In [1]:
import numpy as np
import pandas as pd

from warnings import filterwarnings

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.optimizers.schedules import ExponentialDecay

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import RobustScaler, normalize, LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import GroupKFold, KFold, StratifiedKFold

filterwarnings('ignore')

# MODELO EN 40 SEMANAS

In [2]:
CONFIG = {
    'TRAIN_PATH': '/kaggle/input/datathon-entel-2022-reto2/train.csv',
    'TEST_PATH': '/kaggle/input/datathon-entel-2022-reto2/test.csv',
    'SAMPLE_SUBMISSION': '/kaggle/input/datathon-entel-2022-reto2/test_sample.csv'
}

df_train = pd.read_csv(CONFIG['TRAIN_PATH'])
df_test = pd.read_csv(CONFIG['TEST_PATH'])
df_sub = pd.read_csv(CONFIG['SAMPLE_SUBMISSION'])

In [3]:
n = 40
x_base = 80

In [4]:
def featuring(df):
    
    df = pd.concat([df_train.iloc[:, :5], df], axis=1)
        
    df_z_punto_venta = df.groupby(['Z_PUNTO_VENTA'])[df.iloc[:, -20:].columns].transform('max')
    df_z_modelo = df.groupby(['Z_MODELO'])[df.iloc[:, -20:].columns].transform('max')
    df_z_gama = df.groupby(['Z_GAMA'])[df.iloc[:, -20:].columns].transform('max')
    df_z_marca = df.groupby(['Z_MARCA'])[df.iloc[:, -20:].columns].transform('max')
    df_z_departamento = df.groupby(['Z_DEPARTAMENTO'])[df.iloc[:, -20:].columns].transform('max')
    df_z_s_punto_venta = df.groupby(['Z_PUNTO_VENTA'])[df.iloc[:, -20:].columns].transform('sum')
    df_z_s_modelo = df.groupby(['Z_MODELO'])[df.iloc[:, -20:].columns].transform('sum')
    df_z_s_gama = df.groupby(['Z_GAMA'])[df.iloc[:, -20:].columns].transform('sum')
    df_z_s_marca = df.groupby(['Z_MARCA'])[df.iloc[:, -20:].columns].transform('sum')
    df_z_s_departamento = df.groupby(['Z_DEPARTAMENTO'])[df.iloc[:, -20:].columns].transform('sum')
    
    
    df_b_punto_venta = df['Z_PUNTO_VENTA'].apply(lambda x: 1 if x in 
                                                     ['da45328ba820604eb99694768f2a430cd933d161601dcb8491b4a9b555232c59',
                                                      'e1f2d2708f545ddc1d7266ba0cc5ccc88147b77fdf3450e68a974e93018ecf60'] else 0)
    df_b_departameto = df['Z_DEPARTAMENTO'].apply(lambda x: 1 if x  in 
                                                    ['d6c21b948958417ca98b682a573eb8aa1084b292d32f760f253ef53da13e5589'] else 0)
    
    Z_MARCA = df['Z_MARCA'].replace(df['Z_MARCA'].value_counts(normalize=True).to_dict())
    Z_GAMA = df['Z_GAMA'].replace(df['Z_GAMA'].value_counts(normalize=True).to_dict())
    Z_MODELO = df['Z_MODELO'].replace(df['Z_MODELO'].value_counts(normalize=True).to_dict())
    Z_DEPARTAMENTO = df['Z_DEPARTAMENTO'].replace(df['Z_DEPARTAMENTO'].value_counts(normalize=True).to_dict())
    Z_PUNTO_VENTA = df['Z_PUNTO_VENTA'].replace(df['Z_PUNTO_VENTA'].value_counts(normalize=True).to_dict())
        
    df_z = pd.concat([
        df_z_punto_venta,
        df_z_modelo, 
        df_z_gama,
        df_z_marca,
        df_z_departamento,  
        df_z_s_punto_venta,
        df_z_s_modelo,
        df_z_s_gama,
        df_z_s_marca,
        df_z_s_departamento,
        df_b_punto_venta,
        df_b_departameto,
        Z_MARCA,
        Z_GAMA,
        Z_MODELO,
        Z_DEPARTAMENTO,
        Z_PUNTO_VENTA
    ], axis=1).T.reset_index(drop=True).T
    
    df['M_MAX_VALUE'] = df.iloc[:, 5:15].max(axis=1)
    df['M_SUM_VALUE'] = df.iloc[:, 5:15].sum(axis=1)
    df['M_STD_VALUE'] = df.iloc[:, 5:15].std(axis=1)
    df['M_MEAN_VALUE'] = df.iloc[:, 5:15].mean(axis=1)
    df['M_COUNT_VALUE'] = df.iloc[:, 5:15].stack().apply(lambda x: x if x > 0 else np.nan).unstack(level=1).count(axis=1)
    
    df['M_MAX_VALUE_1'] = df.iloc[:, 15:25].max(axis=1)
    df['M_SUM_VALUE_1'] = df.iloc[:, 15:25].sum(axis=1)
    df['M_STD_VALUE_1'] = df.iloc[:, 15:25].std(axis=1)
    df['M_MEAN_VALUE_1'] = df.iloc[:, 15:25].mean(axis=1)
    df['M_COUNT_VALUE_1'] = df.iloc[:, 15:25].stack().apply(lambda x: x if x > 0 else np.nan).unstack(level=1).count(axis=1)
    
    df['M_MAX_VALUE_2'] = df.iloc[:, 25:35].max(axis=1)
    df['M_SUM_VALUE_2'] = df.iloc[:, 25:35].sum(axis=1)
    df['M_STD_VALUE_2'] = df.iloc[:, 25:35].std(axis=1)
    df['M_MEAN_VALUE_2'] = df.iloc[:, 25:35].mean(axis=1)
    df['M_COUNT_VALUE_2'] = df.iloc[:, 25:35].stack().apply(lambda x: x if x > 0 else np.nan).unstack(level=1).count(axis=1)
    
    df['M_MAX_VALUE_3'] = df.iloc[:, 35:45].max(axis=1)
    df['M_SUM_VALUE_3'] = df.iloc[:, 35:45].sum(axis=1)
    df['M_STD_VALUE_3'] = df.iloc[:, 35:45].std(axis=1)
    df['M_MEAN_VALUE_3'] = df.iloc[:, 35:45].mean(axis=1)
    df['M_COUNT_VALUE_3'] = df.iloc[:, 35:45].stack().apply(lambda x: x if x > 0 else np.nan).unstack(level=1).count(axis=1)
    
    df['M_TOTAL_VALUE'] = df['M_SUM_VALUE'].apply(lambda x: 1 if x > 0 else 0)
    df['M_TOTAL_VALUE_1'] = df['M_SUM_VALUE_1'].apply(lambda x: 1 if x > 0 else 0)
    df['M_TOTAL_VALUE_2'] = df['M_SUM_VALUE_2'].apply(lambda x: 1 if x > 0 else 0)
    df['M_TOTAL_VALUE_3'] = df['M_SUM_VALUE_3'].apply(lambda x: 1 if x > 0 else 0)
        

    df['M_COUNT'] = df.iloc[:, 5:45].stack().apply(lambda x: x if x > 0 else np.nan).unstack(level=1).count(axis=1)
    df[df.iloc[:, 5:45].columns] = df.iloc[:, 5:45].stack().apply(lambda x: x if x < x_base else x_base).unstack(level=1)
        
    df = df.drop(df.columns[:5], axis=1)
    
    df = pd.concat([df, df_z], axis=1)
    
    return df.T.reset_index(drop=True).T

In [5]:
x_train = featuring(df_train.iloc[:, 5:45])
y_train = df_train.iloc[:, 45:55]

x_test = featuring(df_train.iloc[:, 15:55])

In [6]:
print(f'X TRAIN SHAPE: {x_train.shape}')
print(f'Y TRAIN SHAPE: {y_train.shape}')
print(f'X TEST SHAPE: {x_test.shape}')

X TRAIN SHAPE: (47173, 272)
Y TRAIN SHAPE: (47173, 10)
X TEST SHAPE: (47173, 272)


In [7]:
def data_to_models(n_t):
    
    correlated_features = set()
    correlation_matrix = x_train.iloc[:, n_t:].corr()

    for i in range(len(correlation_matrix .columns)):
        for j in range(i):
            if abs(correlation_matrix.iloc[i, j]) > 0.95:
                colname = correlation_matrix.columns[i]
                correlated_features.add(colname)
                
    x_train_model = x_train.drop(labels=correlated_features, axis=1)
    x_test_model = x_test.drop(labels=correlated_features, axis=1)

    print(f'TRAIN SHAPE: {x_train_model.shape}')
    
    sc = StandardScaler()

    x_train_model = sc.fit_transform(x_train_model)
    x_test_model = sc.transform(x_test_model)
    
    return x_train_model, x_test_model

In [8]:
x_train_lstm, x_test_lstm = data_to_models(n)

y_train = y_train.stack().apply(lambda x: x if x < x_base else x_base).unstack(level=1)
y_train = y_train.values

TRAIN SHAPE: (47173, 93)


In [9]:
def LSTM(l_features, l_extras_features):
    
    features = keras.layers.Input(shape=l_features)
    tabular = keras.layers.Input(shape=l_extras_features)

    out_features = keras.layers.LSTM(250, return_sequences=True)(features)
    out_features = keras.layers.Dropout(0.2)(out_features)
    out_features = keras.layers.LSTM(150, return_sequences=True)(out_features)
    out_features = keras.layers.Dropout(0.2)(out_features)
    out_features = keras.layers.LSTM(100)(out_features)
    out_features = keras.layers.Flatten()(out_features)

    out_features = keras.layers.Dense(50, activation='linear')(out_features)
    out_features = keras.layers.Dropout(0.2)(out_features)
    out_features = keras.layers.Dense(32, activation='linear')(out_features)

    for n_hidden in [128, 64, 32]:
        out_tabular = keras.layers.Dense(n_hidden, activation='relu')(tabular)
        out_tabular = keras.layers.BatchNormalization()(out_tabular)
        out_tabular = keras.layers.Dropout(0.2)(out_tabular)

    out = tf.keras.layers.Multiply()([out_features, out_tabular])
    out = keras.layers.Dense(10, activation='relu')(out)

    model = keras.Model(inputs = [features, tabular], outputs = out)

    mse = tf.keras.losses.MeanSquaredError()
    rmse = tf.keras.metrics.RootMeanSquaredError()
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0004), loss=mse, metrics=[rmse])

    return model

In [10]:
def training_LSTM():

    EPOCH = 1000
    BATCH_SIZE = 512

    models = []

    kf = KFold(n_splits=5, shuffle=True, random_state=2022)

    for fold, (train_idx, val_idx) in enumerate(kf.split(x_train_lstm, y_train)):

        print('-'*15, '>', f'Fold {fold+1}', '<', '-'*15)

        X_train, X_valid = x_train_lstm[train_idx], x_train_lstm[val_idx]
        
        x_t_features, x_v_features = X_train[:, :n], X_valid[:, :n]
        x_t_features = np.reshape(x_t_features, (x_t_features.shape[0], 1, x_t_features.shape[1]))
        x_v_features = np.reshape(x_v_features, (x_v_features.shape[0], 1, x_v_features.shape[1]))
        
        x_t_extras, x_v_extras = X_train[:, n:], X_valid[:, n:]
        
        Y_train, Y_valid = y_train[train_idx], y_train[val_idx]

        l_fet = x_t_features.shape[-2:]
        l_ext = x_t_extras.shape[-1]

        model = LSTM(l_fet, l_ext)

        es = keras.callbacks.EarlyStopping(monitor='val_root_mean_squared_error', 
                                           min_delta=1e-05,
                                           patience=30,
                                           verbose=1, 
                                           mode='min', 
                                           restore_best_weights=True)
        plateau = keras.callbacks.ReduceLROnPlateau(monitor='val_root_mean_squared_error',
                                                    factor=0.1,
                                                    patience=10,
                                                    verbose=1,
                                                    min_lr=5e-7, 
                                                    mode='min')

        model.fit([x_t_features, x_t_extras], Y_train,
                  validation_data=([x_v_features, x_v_extras], Y_valid),
                  epochs=EPOCH,
                  batch_size=BATCH_SIZE,
                  callbacks = [es, plateau],
                  verbose=1)

        eval_model(model, [x_v_features, x_v_extras], Y_valid)

        models.append(model)

    return models

def eval_model(model, x_valid, y_valid):
    preds = pd.DataFrame(np.round(model.predict(x_valid)).astype('int32')).stack().reset_index(drop=True)
    y_valid = pd.DataFrame(y_valid).stack().reset_index(drop=True)
    print(f' RMSE --> {mean_squared_error(y_valid, preds, squared=False)}')

In [11]:
models_LSTM_40 = training_LSTM()

--------------- > Fold 1 < ---------------


2022-08-08 17:43:36.121923: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-08 17:43:36.225250: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-08 17:43:36.226021: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-08 17:43:36.227617: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Epoch 1/1000


2022-08-08 17:43:45.013595: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


74/74 [==============================] - 8s 23ms/step - loss: 9.8547 - root_mean_squared_error: 3.1392 - val_loss: 6.4513 - val_root_mean_squared_error: 2.5399
Epoch 2/1000
74/74 [==============================] - 1s 8ms/step - loss: 7.3245 - root_mean_squared_error: 2.7064 - val_loss: 5.0521 - val_root_mean_squared_error: 2.2477
Epoch 3/1000
74/74 [==============================] - 1s 8ms/step - loss: 5.9026 - root_mean_squared_error: 2.4295 - val_loss: 4.1301 - val_root_mean_squared_error: 2.0323
Epoch 4/1000
74/74 [==============================] - 1s 8ms/step - loss: 5.0151 - root_mean_squared_error: 2.2394 - val_loss: 3.8602 - val_root_mean_squared_error: 1.9647
Epoch 5/1000
74/74 [==============================] - 1s 8ms/step - loss: 4.7081 - root_mean_squared_error: 2.1698 - val_loss: 3.6093 - val_root_mean_squared_error: 1.8998
Epoch 6/1000
74/74 [==============================] - 1s 8ms/step - loss: 4.2852 - root_mean_squared_error: 2.0701 - val_loss: 3.1368 - val_root_mean_sq

In [12]:
preds_LSTM_40 = []

x_test_features = x_test_lstm[:, :n]
x_test_features = np.reshape(x_test_features, (x_test_features.shape[0], 1, x_test_features.shape[1]))

x_test_extras = x_test_lstm[:, n:]

for model in models_LSTM_40:
    _pred = model.predict([x_test_features, x_test_extras])
    preds_LSTM_40.append(_pred)
    
pred_sub_LSTM_40 = (preds_LSTM_40[0] + preds_LSTM_40[1] + preds_LSTM_40[2] + preds_LSTM_40[3] + preds_LSTM_40[4]) / 5

# MODELO EN 20 SEMANAS

In [13]:
n = 20

In [14]:
def featuring(df):
    
    df = pd.concat([df_train.iloc[:, :5], df], axis=1)
        
    df_z_punto_venta = df.groupby(['Z_PUNTO_VENTA'])[df.iloc[:, 5: 5 + n].columns].transform('max')
    df_z_modelo = df.groupby(['Z_MODELO'])[df.iloc[:, 5: 5 + n].columns].transform('max')
    df_z_gama = df.groupby(['Z_GAMA'])[df.iloc[:, 5: 5 + n].columns].transform('max')
    df_z_marca = df.groupby(['Z_MARCA'])[df.iloc[:, 5: 5 + n].columns].transform('max')
    df_z_departamento = df.groupby(['Z_DEPARTAMENTO'])[df.iloc[:, 5: 5 + n].columns].transform('max')
    df_z_s_punto_venta = df.groupby(['Z_PUNTO_VENTA'])[df.iloc[:, 5: 5 + n].columns].transform('sum')
    df_z_s_modelo = df.groupby(['Z_MODELO'])[df.iloc[:, 5: 5 + n].columns].transform('sum')
    df_z_s_gama = df.groupby(['Z_GAMA'])[df.iloc[:, 5: 5 + n].columns].transform('sum')
    df_z_s_marca = df.groupby(['Z_MARCA'])[df.iloc[:, 5: 5 + n].columns].transform('sum')
    df_z_s_departamento = df.groupby(['Z_DEPARTAMENTO'])[df.iloc[:, 5: 5 + n].columns].transform('sum')
    
    df_b_punto_venta = df['Z_PUNTO_VENTA'].apply(lambda x: 1 if x in 
                                                     ['da45328ba820604eb99694768f2a430cd933d161601dcb8491b4a9b555232c59',
                                                      'e1f2d2708f545ddc1d7266ba0cc5ccc88147b77fdf3450e68a974e93018ecf60'] else 0)
    df_b_departameto = df['Z_DEPARTAMENTO'].apply(lambda x: 1 if x  in 
                                                    ['d6c21b948958417ca98b682a573eb8aa1084b292d32f760f253ef53da13e5589'] else 0)
    
    Z_MARCA = df['Z_MARCA'].replace(df['Z_MARCA'].value_counts(normalize=True).to_dict())
    Z_GAMA = df['Z_GAMA'].replace(df['Z_GAMA'].value_counts(normalize=True).to_dict())
    Z_MODELO = df['Z_MODELO'].replace(df['Z_MODELO'].value_counts(normalize=True).to_dict())
    Z_DEPARTAMENTO = df['Z_DEPARTAMENTO'].replace(df['Z_DEPARTAMENTO'].value_counts(normalize=True).to_dict())
    Z_PUNTO_VENTA = df['Z_PUNTO_VENTA'].replace(df['Z_PUNTO_VENTA'].value_counts(normalize=True).to_dict())
        
    df_z = pd.concat([df_z_punto_venta,
                      df_z_modelo, 
                      df_z_gama,
                      df_z_marca,
                      df_z_departamento,
                      
                      df_z_s_punto_venta,
                      df_z_s_modelo,
                      df_z_s_gama,
                      df_z_s_marca,
                      df_z_s_departamento,

                      df_b_punto_venta,
                      df_b_departameto,
                      
                      Z_MARCA,
                      Z_GAMA,
                      Z_MODELO,
                      Z_DEPARTAMENTO,
                      Z_PUNTO_VENTA
                     ], axis=1).T.reset_index(drop=True).T
    
    df['M_MAX_VALUE'] = df.iloc[:, 5: 5 + n].max(axis=1)
    df['M_SUM_VALUE'] = df.iloc[:, 5: 5 + n].sum(axis=1)
    df['M_STD_VALUE'] = df.iloc[:, 5: 5 + n].std(axis=1)
    df['M_MEAN_VALUE'] = df.iloc[:, 5: 5 + n].mean(axis=1)
    
    df['M_TOTAL_VALUE'] = df['M_SUM_VALUE'].apply(lambda x: 1 if x > 0 else 0)
    
    df['M_COUNT'] = df.iloc[:, 5: 5 + n].stack().apply(lambda x: x if x > 0 else np.nan).unstack(level=1).count(axis=1)
    
    df[df.iloc[:, 5: 5 + n].columns] = df.iloc[:, 5: 5 + n].stack().apply(lambda x: x if x < x_base else x_base).unstack(level=1)
        
    df = df.drop(df.columns[:5], axis=1)
    
    df = pd.concat([df, df_z], axis=1)
    
    return df.T.reset_index(drop=True).T

In [15]:
x_train = pd.concat([
    featuring(df_train.iloc[:, 5:25]),
    featuring(df_train.iloc[:, 15:35]),
    featuring(df_train.iloc[:, 25:45]),
], axis=0).reset_index(drop=True)

y_train = pd.concat([
    pd.DataFrame(df_train.iloc[:, 25:35].values),
    pd.DataFrame(df_train.iloc[:, 35:45].values),
    pd.DataFrame(df_train.iloc[:, 45:55].values),
], axis=0).reset_index(drop=True)


x_test = featuring(df_train.iloc[:, 35:55])

In [16]:
print(f'X TRAIN SHAPE: {x_train.shape}')
print(f'Y TRAIN SHAPE: {y_train.shape}')
print(f'X TEST SHAPE: {x_test.shape}')

X TRAIN SHAPE: (141519, 233)
Y TRAIN SHAPE: (141519, 10)
X TEST SHAPE: (47173, 233)


In [17]:
def data_to_models(n_t):
    
    correlated_features = set()
    correlation_matrix = x_train.iloc[:, n_t:].corr()

    for i in range(len(correlation_matrix .columns)):
        for j in range(i):
            if abs(correlation_matrix.iloc[i, j]) > 0.95:
                colname = correlation_matrix.columns[i]
                correlated_features.add(colname)
                
    x_train_model = x_train.drop(labels=correlated_features, axis=1)
    x_test_model = x_test.drop(labels=correlated_features, axis=1)

    print(f'TRAIN SHAPE: {x_train_model.shape}')
    
    x_train_model = x_train
    x_test_model = x_test
    
    sc = StandardScaler()

    x_train_model = sc.fit_transform(x_train_model)
    x_test_model = sc.transform(x_test_model)
    
    return x_train_model, x_test_model

In [18]:
x_train_lstm, x_test_lstm = data_to_models(n)

y_train = y_train.stack().apply(lambda x: x if x < x_base else x_base).unstack(level=1).values

TRAIN SHAPE: (141519, 103)


In [19]:
models_LSTM_20 = training_LSTM()

--------------- > Fold 1 < ---------------
Epoch 1/1000
222/222 [==============================] - 6s 14ms/step - loss: 10.0604 - root_mean_squared_error: 3.1718 - val_loss: 7.0326 - val_root_mean_squared_error: 2.6519
Epoch 2/1000
222/222 [==============================] - 2s 8ms/step - loss: 7.2185 - root_mean_squared_error: 2.6867 - val_loss: 6.1366 - val_root_mean_squared_error: 2.4772
Epoch 3/1000
222/222 [==============================] - 2s 9ms/step - loss: 6.5173 - root_mean_squared_error: 2.5529 - val_loss: 5.8425 - val_root_mean_squared_error: 2.4171
Epoch 4/1000
222/222 [==============================] - 2s 8ms/step - loss: 6.2771 - root_mean_squared_error: 2.5054 - val_loss: 5.6809 - val_root_mean_squared_error: 2.3835
Epoch 5/1000
222/222 [==============================] - 2s 8ms/step - loss: 5.9217 - root_mean_squared_error: 2.4335 - val_loss: 5.3772 - val_root_mean_squared_error: 2.3189
Epoch 6/1000
222/222 [==============================] - 2s 8ms/step - loss: 5.6398 - 

In [20]:
preds_LSTM_20 = []

x_test_features = x_test_lstm[:, :n]
x_test_features = np.reshape(x_test_features, (x_test_features.shape[0], 1, x_test_features.shape[1]))

x_test_extras = x_test_lstm[:, n:]

for model in models_LSTM_20:
    _pred = model.predict([x_test_features, x_test_extras])
    preds_LSTM_20.append(_pred)
    
pred_sub_LSTM_20 = (preds_LSTM_20[0] + preds_LSTM_20[1] + preds_LSTM_20[2] + preds_LSTM_20[3] + preds_LSTM_20[4]) / 5

# MODELO BASADO EN 10 SEMANAS

In [21]:
n = 10

In [22]:
def featuring(df):
    
    df = pd.concat([df_train.iloc[:, :5], df], axis=1)
        
    df_z_punto_venta = df.groupby(['Z_PUNTO_VENTA'])[df.iloc[:, 5:].columns].transform('max')
    df_z_modelo = df.groupby(['Z_MODELO'])[df.iloc[:, 5:].columns].transform('max')
    df_z_gama = df.groupby(['Z_GAMA'])[df.iloc[:, 5:].columns].transform('max')
    df_z_marca = df.groupby(['Z_MARCA'])[df.iloc[:, 5:].columns].transform('max')
    df_z_departamento = df.groupby(['Z_DEPARTAMENTO'])[df.iloc[:, 5:].columns].transform('max')
    df_z_s_punto_venta = df.groupby(['Z_PUNTO_VENTA'])[df.iloc[:, 5:].columns].transform('sum')
    df_z_s_modelo = df.groupby(['Z_MODELO'])[df.iloc[:, 5:].columns].transform('sum')
    df_z_s_gama = df.groupby(['Z_GAMA'])[df.iloc[:, 5:].columns].transform('sum')
    df_z_s_marca = df.groupby(['Z_MARCA'])[df.iloc[:, 5:].columns].transform('sum')
    df_z_s_departamento = df.groupby(['Z_DEPARTAMENTO'])[df.iloc[:, 5:].columns].transform('sum')
    
    df_b_punto_venta = df['Z_PUNTO_VENTA'].apply(lambda x: 1 if x in 
                                                     ['da45328ba820604eb99694768f2a430cd933d161601dcb8491b4a9b555232c59',
                                                      'e1f2d2708f545ddc1d7266ba0cc5ccc88147b77fdf3450e68a974e93018ecf60'] else 0)
    df_b_departameto = df['Z_DEPARTAMENTO'].apply(lambda x: 1 if x  in 
                                                    ['d6c21b948958417ca98b682a573eb8aa1084b292d32f760f253ef53da13e5589'] else 0)
    
    Z_MARCA = df['Z_MARCA'].replace(df['Z_MARCA'].value_counts(normalize=True).to_dict())
    Z_GAMA = df['Z_GAMA'].replace(df['Z_GAMA'].value_counts(normalize=True).to_dict())
    Z_MODELO = df['Z_MODELO'].replace(df['Z_MODELO'].value_counts(normalize=True).to_dict())
    Z_DEPARTAMENTO = df['Z_DEPARTAMENTO'].replace(df['Z_DEPARTAMENTO'].value_counts(normalize=True).to_dict())
    Z_PUNTO_VENTA = df['Z_PUNTO_VENTA'].replace(df['Z_PUNTO_VENTA'].value_counts(normalize=True).to_dict())
        
    df_z = pd.concat([df_z_punto_venta,
                      df_z_modelo, 
                      df_z_gama,
                      df_z_marca,
                      df_z_departamento,
                      
                      df_z_s_punto_venta,
                      df_z_s_modelo,
                      df_z_s_gama,
                      df_z_s_marca,
                      df_z_s_departamento,

                      df_b_punto_venta,
                      df_b_departameto,
                      
                      Z_MARCA,
                      Z_GAMA,
                      Z_MODELO,
                      Z_DEPARTAMENTO,
                      Z_PUNTO_VENTA
                     ], axis=1).T.reset_index(drop=True).T
    
    df['M_MAX_VALUE'] = df.iloc[:, 5:15].max(axis=1)
    df['M_SUM_VALUE'] = df.iloc[:, 5:15].sum(axis=1)
    df['M_STD_VALUE'] = df.iloc[:, 5:15].std(axis=1)
    df['M_MEAN_VALUE'] = df.iloc[:, 5:15].mean(axis=1)
    
    df['M_TOTAL_VALUE'] = df['M_SUM_VALUE'].apply(lambda x: 1 if x > 0 else 0)
    
    df['M_COUNT'] = df.iloc[:, 5:15].stack().apply(lambda x: x if x > 0 else np.nan).unstack(level=1).count(axis=1)
    
    df[df.iloc[:, 5:15].columns] = df.iloc[:, 5:15].stack().apply(lambda x: x if x < x_base else x_base).unstack(level=1)
        
    df = df.drop(df.columns[:5], axis=1)
    
    df = pd.concat([df, df_z], axis=1)
    
    return df.T.reset_index(drop=True).T

In [23]:
x_train = pd.concat([
    featuring(df_train.iloc[:, 5:15]),
    featuring(df_train.iloc[:, 15:25]),
    featuring(df_train.iloc[:, 25:35]),
    featuring(df_train.iloc[:, 35:45]),
], axis=0).reset_index(drop=True)

y_train = pd.concat([
    pd.DataFrame(df_train.iloc[:, 15:25].values),
    pd.DataFrame(df_train.iloc[:, 25:35].values),
    pd.DataFrame(df_train.iloc[:, 35:45].values),
    pd.DataFrame(df_train.iloc[:, 45:55].values)
], axis=0).reset_index(drop=True)


x_test = featuring(df_train.iloc[:, 45:55])

In [24]:
print(f'X TRAIN SHAPE: {x_train.shape}')
print(f'Y TRAIN SHAPE: {y_train.shape}')
print(f'X TEST SHAPE: {x_test.shape}')

X TRAIN SHAPE: (188692, 123)
Y TRAIN SHAPE: (188692, 10)
X TEST SHAPE: (47173, 123)


In [25]:
def data_to_models(n_t):
    
    correlated_features = set()
    correlation_matrix = x_train.iloc[:, n_t:].corr()

    for i in range(len(correlation_matrix .columns)):
        for j in range(i):
            if abs(correlation_matrix.iloc[i, j]) > 0.95:
                colname = correlation_matrix.columns[i]
                correlated_features.add(colname)
                
    x_train_model = x_train.drop(labels=correlated_features, axis=1)
    x_test_model = x_test.drop(labels=correlated_features, axis=1)

    print(f'TRAIN SHAPE: {x_train_model.shape}')
    
    x_train_model = x_train
    x_test_model = x_test
    
    sc = StandardScaler()

    x_train_model = sc.fit_transform(x_train_model)
    x_test_model = sc.transform(x_test_model)
    
    return x_train_model, x_test_model

In [26]:
x_train_lstm, x_test_lstm = data_to_models(n)

y_train = y_train.stack().apply(lambda x: x if x < x_base else x_base).unstack(level=1).values

TRAIN SHAPE: (188692, 62)


In [27]:
models_LSTM_10 = training_LSTM()

--------------- > Fold 1 < ---------------
Epoch 1/1000
295/295 [==============================] - 7s 13ms/step - loss: 9.2934 - root_mean_squared_error: 3.0485 - val_loss: 7.3119 - val_root_mean_squared_error: 2.7041
Epoch 2/1000
295/295 [==============================] - 3s 9ms/step - loss: 7.1349 - root_mean_squared_error: 2.6711 - val_loss: 5.9916 - val_root_mean_squared_error: 2.4478
Epoch 3/1000
295/295 [==============================] - 2s 8ms/step - loss: 6.6721 - root_mean_squared_error: 2.5830 - val_loss: 5.7020 - val_root_mean_squared_error: 2.3879
Epoch 4/1000
295/295 [==============================] - 2s 8ms/step - loss: 6.5070 - root_mean_squared_error: 2.5509 - val_loss: 5.7198 - val_root_mean_squared_error: 2.3916
Epoch 5/1000
295/295 [==============================] - 2s 8ms/step - loss: 6.2507 - root_mean_squared_error: 2.5001 - val_loss: 5.3512 - val_root_mean_squared_error: 2.3133
Epoch 6/1000
295/295 [==============================] - 2s 8ms/step - loss: 6.0804 - r

In [28]:
preds_LSTM_10 = []

x_test_features = x_test_lstm[:, :n]
x_test_features = np.reshape(x_test_features, (x_test_features.shape[0], 1, x_test_features.shape[1]))

x_test_extras = x_test_lstm[:, n:]

for model in models_LSTM_10:
    _pred = model.predict([x_test_features, x_test_extras])
    preds_LSTM_10.append(_pred)
    
pred_sub_LSTM_10 = (preds_LSTM_10[0] + preds_LSTM_10[1] + preds_LSTM_10[2] + preds_LSTM_10[3] + preds_LSTM_10[4]) / 5

# INTEGRACIÓN DE MODELOS

In [29]:
pred_sub = pred_sub_LSTM_40 * 0.4 + pred_sub_LSTM_20 * 0.3 + pred_sub_LSTM_10 * 0.3

In [30]:
df_submission = pd.merge(df_train.iloc[:, :5], pd.DataFrame(pred_sub), how='inner', left_index=True, right_index=True)
df_submission = df_submission.rename(columns={
    0: 'SEMANA_51', 
    1: 'SEMANA_52',
    2: 'SEMANA_53',
    3: 'SEMANA_54',
    4: 'SEMANA_55',
    5: 'SEMANA_56',
    6: 'SEMANA_57',
    7: 'SEMANA_58',
    8: 'SEMANA_59',
    9: 'SEMANA_60'
})

df_submission['BASE_ID'] = df_submission['Z_MODELO'].astype(str) + '|' + df_submission['Z_PUNTO_VENTA'].astype(str) + '|' + df_submission['Z_GAMA'].astype(str)
df_submission = df_submission.iloc[:, 5:]
df_submission = df_submission.set_index('BASE_ID').stack().to_frame().reset_index()
df_submission['BASE_ID'] = df_submission['BASE_ID'].astype(str) + '|' + df_submission['level_1'].astype(str)
df_submission = df_submission.drop(['level_1'], axis=1)
df_submission.columns = ['ID', 'Demanda']

In [31]:
df_submission.to_csv('entel_v1_1_2.csv', index=False)

In [32]:
p = 0.9999
print(df_submission.Demanda.quantile(p))
print(pd.DataFrame(y_train).stack().quantile(p))
# 76.832

56.96454716565741
80.0


<a href='./entel_v1_1_2.csv'>download</a>